In [72]:
%load_ext autoreload
%autoreload 2

import data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
houses_data = data.Data('../dataset/test.csv', '../dataset/train.csv')
houses_data.handle_missing_data()
houses_data.preprocessing_data()
houses_data.remove_duplicate_features()

/Users/ngohongthai/Documents/projects/tensorflow-env/env/lib/python3.8/site-packages/feature_engine/encoding/one_hot.py:248: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[str(feature) + "_" + str(category)] = np.where(
/Users/ngohongthai/Documents/projects/tensorflow-env/env/lib/python3.8/site-packages/feature_engine/encoding/one_hot.py:248: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[str(feature) + "_" + str(category)] = np.where(
/Users/ngohongthai/Documents/projects/tensorflow-env/env/lib/python3.8/site-packages/feature

In [74]:
train_lasso_selected, test_lasso_selected = houses_data.select_features_by_lasso()
train_recursive_ellimination_selected, test_recursive_ellimination_selected = houses_data.select_features_by_recursive_feature_elimination()
train_recursive_addition_selected, test_recursive_addition_selected = houses_data.select_features_by_recursive_feature_addition()

/Users/ngohongthai/Documents/projects/tensorflow-env/env/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/Users/ngohongthai/Documents/projects/tensorflow-env/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale 

## Evaluation

In [75]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [76]:
# Số fold bằng 10
n_folds = 10
y = houses_data.target_train

def rmse_cv(model, train, y):
  
    # Tạo danh sách các fold
    kf = KFold(n_folds, shuffle=True, random_state=0).get_n_splits(train.values)

    # Tiến hành kiểm chứng chéo với metric là MSE
    mse= np.sqrt(-cross_val_score(model, train.values, np.log(y), scoring="neg_mean_squared_error", cv = kf))

    # Trả về mảng giá trị MSE của từng fold
    return(mse)

In [77]:
model = Ridge(alpha = 1e+3, tol = 0.0001, random_state=0)

score = rmse_cv(model, train_lasso_selected, y)
print("Lasso Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_ellimination_selected, y)
print("Recursive Ellimination Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_addition_selected, y)
print("Recursive Addition Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso Selection: Averaged base models score: 0.1676 (0.0338)

Recursive Ellimination Selection: Averaged base models score: 0.1677 (0.0343)

Recursive Addition Selection: Averaged base models score: 0.2058 (0.0279)



In [78]:
model = ElasticNet(alpha = 8, l1_ratio = 0.01, tol = 0.0001, random_state=0)

## PUT YOUR CODE HERE:
# Kiểm chứng chéo trên các tập dữ liệu đã được lựa chọn thuộc tính

score = rmse_cv(model, train_lasso_selected, y)
print("Lasso Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_ellimination_selected, y)
print("Recursive Ellimination Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_addition_selected, y)
print("Recursive Addition Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso Selection: Averaged base models score: 0.3019 (0.0233)

Recursive Ellimination Selection: Averaged base models score: 0.3042 (0.0235)

Recursive Addition Selection: Averaged base models score: 0.3480 (0.0256)



In [79]:
model = SVR(kernel = 'sigmoid')

## PUT YOUR CODE HERE:
# Kiểm chứng chéo trên các tập dữ liệu đã được lựa chọn thuộc tính

score = rmse_cv(model, train_lasso_selected, y)
print("Lasso Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_ellimination_selected, y)
print("Recursive Ellimination Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_addition_selected, y)
print("Recursive Addition Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso Selection: Averaged base models score: 4.8789 (0.6320)

Recursive Ellimination Selection: Averaged base models score: 4.1538 (0.5859)

Recursive Addition Selection: Averaged base models score: 25.1777 (3.0435)



In [80]:
model = DecisionTreeRegressor(random_state = 0)

## PUT YOUR CODE HERE:
# Kiểm chứng chéo trên các tập dữ liệu đã được lựa chọn thuộc tính

score = rmse_cv(model, train_lasso_selected, y)
print("Lasso Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_ellimination_selected, y)
print("Recursive Ellimination Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_addition_selected, y)
print("Recursive Addition Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso Selection: Averaged base models score: 0.1959 (0.0247)

Recursive Ellimination Selection: Averaged base models score: 0.2042 (0.0170)

Recursive Addition Selection: Averaged base models score: 0.2132 (0.0227)



In [81]:
model = RandomForestRegressor(n_estimators = 10, random_state = 0)

## PUT YOUR CODE HERE:
# Kiểm chứng chéo trên các tập dữ liệu đã được lựa chọn thuộc tính

score = rmse_cv(model, train_lasso_selected, y)
print("Lasso Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_ellimination_selected, y)
print("Recursive Ellimination Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmse_cv(model, train_recursive_addition_selected, y)
print("Recursive Addition Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso Selection: Averaged base models score: 0.1487 (0.0201)

Recursive Ellimination Selection: Averaged base models score: 0.1492 (0.0213)

Recursive Addition Selection: Averaged base models score: 0.1613 (0.0149)



## Tìm kiếm tham số tối ưu

In [82]:
parameters = {'n_estimators': [10, 20, 50, 100]}

# Khởi tạo lưới tìm kiếm trên mô hình đã được lựa chọn
# trên tập dữ liệu loại đặc trưng bằng Lasso
model_lasso = RandomForestRegressor(random_state = 0)
grid_lasso = GridSearchCV(model_lasso, parameters , scoring = 'neg_mean_squared_error')
grid_lasso.fit(train_lasso_selected, y)

GridSearchCV(estimator=RandomForestRegressor(random_state=0),
             param_grid={'n_estimators': [10, 20, 50, 100]},
             scoring='neg_mean_squared_error')

In [83]:
grid_lasso.best_params_

{'n_estimators': 100}

In [84]:
parameters = {'n_estimators': [10, 20, 50, 100]}

# Khởi tạo lưới tìm kiếm trên mô hình đã được lựa chọn
# trên tập dữ liệu loại đặc trưng bằng đệ quy
model_recursive_ellimination = RandomForestRegressor(random_state = 0)
grid_recursive_ellimination = GridSearchCV(model_recursive_ellimination, parameters, scoring = 'neg_mean_squared_error')
grid_recursive_ellimination.fit(train_recursive_ellimination_selected, y)

GridSearchCV(estimator=RandomForestRegressor(random_state=0),
             param_grid={'n_estimators': [10, 20, 50, 100]},
             scoring='neg_mean_squared_error')

In [85]:
grid_recursive_ellimination.best_params_

{'n_estimators': 100}

In [86]:
parameters = {'n_estimators': [10, 20, 50, 100]}

# Khởi tạo lưới tìm kiếm trên mô hình đã được lựa chọn 
# trên tập dữ liệu thêm đặc trưng bằng đệ quy
model_recursive_addition = RandomForestRegressor(random_state = 0)
grid_recursive_addition = GridSearchCV(model_recursive_addition, parameters, scoring = 'neg_mean_squared_error')
grid_recursive_addition.fit(train_recursive_addition_selected, y)

GridSearchCV(estimator=RandomForestRegressor(random_state=0),
             param_grid={'n_estimators': [10, 20, 50, 100]},
             scoring='neg_mean_squared_error')

In [87]:
grid_recursive_addition.best_params_

{'n_estimators': 100}

In [88]:
model_lasso = RandomForestRegressor(n_estimators = grid_lasso.best_params_['n_estimators'], random_state = 0)
score = rmse_cv(model_lasso, train_lasso_selected, y)
print("Lasso Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

model_recursive = RandomForestRegressor(n_estimators = grid_recursive_ellimination.best_params_['n_estimators'], random_state = 0)
score = rmse_cv(model_recursive, train_recursive_ellimination_selected, y)
print("Recursive Ellimination Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

model_recursive = RandomForestRegressor(n_estimators = grid_recursive_addition.best_params_['n_estimators'], random_state = 0)
score = rmse_cv(model_recursive, train_recursive_addition_selected, y)
print("Recursive Addition Selection: Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso Selection: Averaged base models score: 0.1418 (0.0182)

Recursive Ellimination Selection: Averaged base models score: 0.1424 (0.0185)

Recursive Addition Selection: Averaged base models score: 0.1557 (0.0137)



In [89]:
model_lasso = RandomForestRegressor(n_estimators = grid_lasso.best_params_['n_estimators'], random_state = 0)
model_lasso.fit(train_lasso_selected, y)
model_lasso.predict(test_lasso_selected)

array([123069.33, 159403.  , 187687.75, ..., 161290.77, 120084.5 ,
       236484.18])

In [90]:
model_ellimination = RandomForestRegressor(n_estimators = grid_recursive_ellimination.best_params_['n_estimators'], random_state = 0)
model_ellimination.fit(train_recursive_ellimination_selected, y)
model_ellimination.predict(test_recursive_ellimination_selected)

array([123112.41, 162812.5 , 190189.33, ..., 168185.43, 121541.25,
       236862.25])

In [91]:
model_addition = RandomForestRegressor(n_estimators = grid_recursive_addition.best_params_['n_estimators'], random_state = 0)
model_addition.fit(train_recursive_addition_selected, y)
model_addition.predict(test_recursive_addition_selected)

array([119945.83, 157791.5 , 194978.2 , ..., 146535.6 , 123754.5 ,
       232439.  ])

In [92]:
# Tạo file dùng để submit
sub = pd.DataFrame()
sub['Id'] = houses_data.test_ids
sub['SalePrice'] = model_lasso.predict(test_lasso_selected)
sub.to_csv('submission_lasso_select.csv',index=False)

In [93]:
# Tạo file dùng để submit
sub = pd.DataFrame()
sub['Id'] = houses_data.test_ids
sub['SalePrice'] = model_ellimination.predict(test_recursive_ellimination_selected)
sub.to_csv('submission_recuresive_ellimination.csv',index=False)

In [94]:
# Tạo file dùng để submit
sub = pd.DataFrame()
sub['Id'] = houses_data.test_ids
sub['SalePrice'] = model_addition.predict(test_recursive_addition_selected)
sub.to_csv('submission_recuresive_addition.csv',index=False)

Điền các score sau khi submit lên kaggle vào ô trống:

- Lasso Selection: `0.14626`
- Recursive Addition Feature: `0.15422`
- Recursive Ellimination Feature: `0.14628`